In [1]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [4]:
cd "drive/My Drive/Colab Notebooks/ML_proj2"

/content/drive/My Drive/Colab Notebooks/ML_proj2


In [5]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image

from keras.preprocessing.image import ImageDataGenerator
from skimage import io


import tensorflow as tf
from tensorflow import keras

from sklearn.feature_extraction import image

#import kerastuner as kt

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow.keras import regularizers

#unet
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import backend as keras

import skimage.transform as trans

from preprocess_data import *

import os
from keras.models import Model, load_model

from metrics import *
import torch

from keras.layers import Input, BatchNormalization, Dropout, LeakyReLU, ReLU

In [ ]:
#to use the GPU
#First go to execution in colab and set the execution type to GPU then run this cell 

if not torch.cuda.is_available():
  raise Exception("Things will go much quicker if you enable a GPU in Colab under 'Runtime / Change Runtime Type'")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [ ]:
#prerocess the data
#load the images and their corresponding groudtruths, do the data augmentation and then split into train and test sets 

#houni hatit wahda lel unet w wahda lel cnn mais selim kali eli hata howa il prefere tkoun andou eli aamaltha nea lel unet donc choufou w badlouha de en fonction chnowa thebou tkhaliw
#khater ena fel unet ma nkasamhech el images el patches taatihelou kemla juste naamel data aug 

#fel preprocess data badalet faza eli heya fel les 100 images lowlin yakra el img wel gt eb nafes el indice khater kbal ken yakrahom mouch bel shih genre yhot el imh maa gt mouch mteeha 
#ma naarafehc alech yaamel hakeka rahou juste salahtha w walet tekhdem

#fel unet el Y naamelelha expansion bech nejem naatiha lel modele rod belek ma thothech lel cnn

#les imports mathabihom tanthifa sghira zeda khater badalet barcha marat donc possible fama hajet maatech netsthakoulhom 

#
root_dir =os.path.join("data", "training1")

X_train,X_test,Y_train,Y_test= preprocess(root_dir, max_iters=20 , random_state=0,
                              test_ratio=0.2, foreground_threshold=0.25)

here 2
unet


In [ ]:
def conv(input, n_filters, kernel_size, activation_Leaky , alpha ):

  #first convolutional layer
  conv1 =  Conv2D(filters=n_filters,
                  kernel_size= kernel_size,
                  kernel_initializer= 'he_normal',
                  padding='same')(input)
  
  #batch normalization
  conv1 = BatchNormalization()(conv1)

  #activation function
  if activation_Leaky:
    conv1= LeakyReLU(alpha)(conv1)
  else: 
    conv1= ReLU(alpha)(conv1)


  #second convolutional layer
  conv2= Conv2D(filters=n_filters, 
                kernel_size= kernel_size, 
                kernel_initializer=  'he_normal',
                padding='same')(conv1)
  
  #batch normalization
  conv2 = BatchNormalization()(conv2)

  #activation function
  if activation_Leaky:
    conv2= LeakyReLU(alpha)(conv2)
  else :
    conv2= ReLU(alpha)(conv2)

  return conv2


In [ ]:
def down_sample(conv , dropout_val ):
    pool = MaxPooling2D(pool_size=(2, 2) ,strides=None, padding='same', data_format='channels_last')(conv)

    if dropout_val != None: 
      pool = Dropout(dropout_val)(pool)

    return pool 


In [ ]:
def up_sample(conv1, conv2, n_filters, kernel_size ): 
  up =Conv2DTranspose(n_filters, kernel_size= kernel_size, strides=(2, 2), padding='same') (conv1)

  merge = concatenate([conv2,up], axis = 3)

  return up, merge


In [ ]:
# U-net model

def unet(img_size, n_channels , n_filters , kernel_size, activation_Leaky, dropout_val, lr , alpha ):
    input_imgs = Input((img_size, img_size , n_channels))

    """
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    """

    #encoder
    #1
    conv1= conv(input_imgs, n_filters*1, kernel_size, activation_Leaky, alpha )
    down1 = down_sample(conv1 , dropout_val)

    #2
    conv2= conv(down1, n_filters*2, kernel_size, activation_Leaky, alpha )
    down2 = down_sample(conv2 , dropout_val)

    #3
    conv3= conv(down2, n_filters*4,  kernel_size, activation_Leaky, alpha  )
    down3 = down_sample(conv3 , dropout_val)

    #4
    conv4= conv(down3, n_filters*8,  kernel_size, activation_Leaky , alpha )
    down4 = down_sample(conv4 , dropout_val)

    #5
    conv5= conv(down4, n_filters*16,  kernel_size, activation_Leaky, alpha  )
    drop5 = Dropout(dropout_val)(conv5)
    #drop5= conv5


    """
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)


    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)
    """

    

    """
    up6 = Conv2D(512, 2, activation , padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    """

    #decoder  
    up6, merge6= up_sample(drop5, conv4,  n_filters*8 , kernel_size) 
    conv6 = conv(merge6, n_filters*8 , kernel_size, activation_Leaky, alpha)

    """
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    """

    """
    up7 = Conv2D(256, 2, activation , padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    """

    up7, merge7= up_sample(conv6, conv3,  n_filters*4 , kernel_size )
    conv7 = conv(merge7, n_filters*4 , kernel_size, activation_Leaky, alpha )

    """
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    """

    """
    up8 = Conv2D(128, 2, activation , padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    """

    up8, merge8= up_sample(conv7, conv2,  n_filters*2 ,kernel_size )
    conv8 = conv(merge8,n_filters*2 , kernel_size, activation_Leaky, alpha )

    """
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    """
    """
    up9 = Conv2D(64, 2, activation , padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    """

    up9, merge9= up_sample(conv8, conv1,  n_filters*1 ,kernel_size )
    conv9 = conv(merge9, n_filters*1 , kernel_size, activation_Leaky, alpha )

    """
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    """

    #conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

    # 1 x 1 convolution
    output = Conv2D(1, 1, activation = 'sigmoid')(conv9)



    model = Model(inputs = [input_imgs], outputs = [output])

    model.compile(optimizer = Adam(), loss='binary_crossentropy', metrics=['acc',f1_m, precision_m, recall_m])
    
    print(model.summary())


    return model



In [ ]:
model =unet(img_size=400, n_channels=3 , n_filters= 16, kernel_size= 3, activation_Leaky=True, dropout_val=0.5 , lr=0, alpha=0.3)

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 400, 400, 3) 0                                            
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 400, 400, 16) 448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_18 (BatchNo (None, 400, 400, 16) 64          conv2d_19[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_18 (LeakyReLU)      (None, 400, 400, 16) 0           batch_normalization_18[0][0]     
_______________________________________________________________________________________

In [ ]:
#callbacks
#nejmou nhotouha fi classe wahadha

model_path= 'saved_model/my_model3'

#hethi thaherli mouch kaada tekhdem
#Callback to save the Keras model or model weights at some frequency.
checkpointer = ModelCheckpoint(model_path,
                             monitor=f1_m,
                             mode="max",
                             save_best_only = True,
                             verbose=1)

#Stop training when f1_m metric has stopped improving for 20 epochs
earlystopper = EarlyStopping(monitor =f1_m, 
                             mode='max', 
                            patience = 20,
                            verbose = 1,
                            restore_best_weights = True)

#Reduce learning rate when loss has stopped improving for 3 epochs
lr_reducer = ReduceLROnPlateau(monitor='loss',
                               mode='min',
                               factor=0.9,
                               patience=3,
                               min_delta= 0.001, 
                               min_lr=0.00001,
                               verbose=1
                               )

metrics =[checkpointer, earlystopper, lr_reducer]

In [ ]:
model.fit(X_train,Y_train, epochs=600, batch_size=16 , callbacks=[metrics])

Epoch 1/600
17/17 [==============================] - 4s 256ms/step - loss: 0.5895 - acc: 0.6123 - f1_m: 0.3897 - precision_m: 0.3163 - recall_m: 0.5208
Epoch 2/600
17/17 [==============================] - 4s 256ms/step - loss: 0.4943 - acc: 0.7226 - f1_m: 0.4742 - precision_m: 0.4598 - recall_m: 0.4949
Epoch 3/600
17/17 [==============================] - 4s 255ms/step - loss: 0.4358 - acc: 0.7453 - f1_m: 0.5422 - precision_m: 0.5312 - recall_m: 0.5646
Epoch 4/600
17/17 [==============================] - 4s 255ms/step - loss: 0.3807 - acc: 0.7716 - f1_m: 0.6104 - precision_m: 0.6057 - recall_m: 0.6216
Epoch 5/600
17/17 [==============================] - 4s 254ms/step - loss: 0.3465 - acc: 0.7884 - f1_m: 0.6350 - precision_m: 0.6523 - recall_m: 0.6198
Epoch 6/600
17/17 [==============================] - 4s 257ms/step - loss: 0.3318 - acc: 0.7916 - f1_m: 0.6395 - precision_m: 0.6681 - recall_m: 0.6200
Epoch 7/600
17/17 [==============================] - 4s 255ms/step - loss: 0.3092 - acc:

In [ ]:
test=model.evaluate(X_test,Y_test)

3/3 [==============================] - 0s 75ms/step - loss: 0.1620 - acc: 0.8431 - f1_m: 0.8527 - precision_m: 0.8405 - recall_m: 0.8654


In [ ]:
#save the model 
print("saving the model")
# Save the entire model as a SavedModel.
model.save('saved_model/my_model3') 

saving the model
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/my_model3/assets


In [ ]:
model.fit(X_train,Y_train, epochs=400, batch_size=16 , callbacks=[metrics])

Epoch 1/400
17/17 [==============================] - 4s 257ms/step - loss: 0.3995 - acc: 0.7658 - f1_m: 0.5112 - precision_m: 0.5753 - recall_m: 0.4667
Epoch 2/400
17/17 [==============================] - 4s 257ms/step - loss: 0.3496 - acc: 0.7828 - f1_m: 0.5991 - precision_m: 0.6378 - recall_m: 0.5694
Epoch 3/400
17/17 [==============================] - 4s 256ms/step - loss: 0.3147 - acc: 0.8009 - f1_m: 0.6428 - precision_m: 0.7028 - recall_m: 0.5945
Epoch 4/400
17/17 [==============================] - 4s 257ms/step - loss: 0.2896 - acc: 0.8064 - f1_m: 0.6774 - precision_m: 0.7169 - recall_m: 0.6453
Epoch 5/400
17/17 [==============================] - 4s 258ms/step - loss: 0.2717 - acc: 0.8119 - f1_m: 0.6958 - precision_m: 0.7343 - recall_m: 0.6639
Epoch 6/400
17/17 [==============================] - 4s 256ms/step - loss: 0.2602 - acc: 0.8170 - f1_m: 0.7055 - precision_m: 0.7618 - recall_m: 0.6608
Epoch 7/400
17/17 [==============================] - 4s 256ms/step - loss: 0.2430 - acc:

In [ ]:
test=model.evaluate(X_test,Y_test)

3/3 [==============================] - 1s 171ms/step - loss: 0.1540 - acc: 0.8437 - f1_m: 0.8587 - precision_m: 0.8608 - recall_m: 0.8585


In [ ]:
model.fit(X_train,Y_train, epochs=100, batch_size=16 , callbacks=[metrics])

Epoch 1/100
11/11 [==============================] - 4s 328ms/step - loss: 0.7777 - acc: 0.3553 - f1_m: 0.3394 - precision_m: 0.2224 - recall_m: 0.7220
Epoch 2/100
11/11 [==============================] - 3s 241ms/step - loss: 0.6322 - acc: 0.5469 - f1_m: 0.3729 - precision_m: 0.2809 - recall_m: 0.5679
Epoch 3/100
11/11 [==============================] - 3s 241ms/step - loss: 0.5586 - acc: 0.6700 - f1_m: 0.3832 - precision_m: 0.3548 - recall_m: 0.4239
Epoch 4/100
11/11 [==============================] - 3s 242ms/step - loss: 0.5112 - acc: 0.7220 - f1_m: 0.3760 - precision_m: 0.4218 - recall_m: 0.3413
Epoch 5/100
11/11 [==============================] - 3s 240ms/step - loss: 0.4818 - acc: 0.7363 - f1_m: 0.3842 - precision_m: 0.4578 - recall_m: 0.3336
Epoch 6/100
11/11 [==============================] - 3s 241ms/step - loss: 0.4547 - acc: 0.7366 - f1_m: 0.4385 - precision_m: 0.4843 - recall_m: 0.4022
Epoch 7/100
11/11 [==============================] - 3s 243ms/step - loss: 0.4268 - acc:

In [ ]:
test=model.evaluate(X_test,Y_test)

2/2 [==============================] - 0s 235ms/step - loss: 0.1692 - acc: 0.8183 - f1_m: 0.8303 - precision_m: 0.8050 - recall_m: 0.8582


In [ ]:
model.fit(X_train,Y_train, epochs=100, batch_size=16 , callbacks=[metrics])

Epoch 1/100
11/11 [==============================] - 222s 20s/step - loss: 0.6555 - acc: 0.5142 - f1_m: 0.2944 - precision_m: 0.2161 - recall_m: 0.4660
Epoch 2/100
11/11 [==============================] - 220s 20s/step - loss: 0.5790 - acc: 0.6495 - f1_m: 0.3091 - precision_m: 0.2748 - recall_m: 0.3611
Epoch 3/100
11/11 [==============================] - 221s 20s/step - loss: 0.5260 - acc: 0.7020 - f1_m: 0.3855 - precision_m: 0.3888 - recall_m: 0.3859
Epoch 4/100
11/11 [==============================] - 221s 20s/step - loss: 0.4907 - acc: 0.7221 - f1_m: 0.4186 - precision_m: 0.4422 - recall_m: 0.4016
Epoch 5/100
11/11 [==============================] - 222s 20s/step - loss: 0.4561 - acc: 0.7423 - f1_m: 0.4625 - precision_m: 0.5083 - recall_m: 0.4276
Epoch 6/100
11/11 [==============================] - 221s 20s/step - loss: 0.4245 - acc: 0.7491 - f1_m: 0.5179 - precision_m: 0.5573 - recall_m: 0.4889
Epoch 7/100
11/11 [==============================] - 223s 20s/step - loss: 0.3946 - acc:

In [ ]:
test=model.evaluate(X_test,Y_test)

In [ ]:
#run the U_net model 

history = model.fit( X_train,Y_train, epochs=60, batch_size=16, verbose=1)

Epoch 1/60
12/12 [==============================] - 280s 23s/step - loss: 0.7069 - precision: 0.2534 - recall: 0.5418
Epoch 2/60
12/12 [==============================] - 280s 23s/step - loss: 0.6206 - precision: 0.3035 - recall: 0.3400
Epoch 3/60
12/12 [==============================] - 281s 23s/step - loss: 0.5705 - precision: 0.3633 - recall: 0.1779
Epoch 4/60
12/12 [==============================] - 281s 23s/step - loss: 0.5382 - precision: 0.4208 - recall: 0.0813
Epoch 5/60
12/12 [==============================] - 281s 23s/step - loss: 0.5206 - precision: 0.4234 - recall: 0.0815
Epoch 6/60
12/12 [==============================] - 280s 23s/step - loss: 0.5068 - precision: 0.4458 - recall: 0.0487
Epoch 7/60
12/12 [==============================] - 280s 23s/step - loss: 0.5011 - precision: 0.4705 - recall: 0.0568
Epoch 8/60
12/12 [==============================] - 280s 23s/step - loss: 0.4904 - precision: 0.4916 - recall: 0.0957
Epoch 9/60
12/12 [==============================] - 280s

In [ ]:
#methode naamlouha lel train, ma kamalthech 
#nzidouha les parametres 
def train_unet():
  model =unet(img_size=400, n_channels=3 , n_filters= 16, kernel_size= 3, activation_Leaky=False, dropout_val=0.5 , lr=0, alpha=0.3)
  history = model.fit( X_train,Y_train, epochs=100, batch_size=16, verbose=1) 
  return model , history